### 获取订单号的参数

```python

qm = QHJMall(MALL_KEY)
qm.get_order(order_type,**kargs)


# 请求类型 0 表示所有订单 1 未发货  2 已发货 3 已完成 4 申请退款 5 退款成功  10 未支付
"order_status": kargs.get("order_status", 0),
"order_time": kargs.get("order_time", ""), # today yesterday week month
"order_id": kargs.get("order_id", ""), # 订单号
"order_type": order_type,  # original free
"page": kargs.get("page", 1),  #

```


In [1]:
import re
import sys
sys.path.append(r"D:\往期\QHJ\ZERO")
sys.path.append(r"E:\dataparse\Python_DATA_PARSE\QHJ\ZERO")
import warnings
warnings.filterwarnings("ignore")

from pyquery import PyQuery as pq
import requests
from urllib.parse import urlencode,parse_qs
import numpy as np
import pandas as pd
import redis

import json
from tools import *

qm = QHJMall(MALL_KEY)


import datetime

sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']


In [28]:
l = []
# for order in qm.gen_orders("free",order_status=1,order_time="today"):
#     l.append(order)
# for order in qm.gen_orders("free",order_status=1,order_time="yesterday"):
#     l.append(order)
for order in qm.gen_orders("original"):
    l.append(order)

请求 {'order_status': 0, 'order_time': '', 'order_id': '', 'order_type': 'original', 'page': 1}


In [31]:
commodity_df = pd.read_excel(COMMODITY_PATH)



# In[3]:

sku_goods = commodity_df \
                [['商品ID', '发货商','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

In [32]:
sku_goods.columns


Index(['商品ID', '发货商', '发货商ID', 'goods_type', '规格', '单位', '市场价', '单价', '商品名'], dtype='object')

In [34]:
data = pd.DataFrame(l)


data = pd.concat([data['用户信息'].str.extract("""
        (?<=下单人：)(?P<收件人>[^\n]+)
        \s+手机：(?P<联系方式>[^\n]+)
        \s+收货地址：(?P<收货地址>[^\n]+)""", flags=re.S | re.X),data],axis=1)
data = data.replace({"支付金额": {"￥": ""}}, regex=True).fillna({"支付金额": 0})

try:
    data['支付金额'] = data['支付金额'].astype(int)
except :
    data['支付金额'] = 0


original_data = data[data['order_type'] == "original"]
# 拆分为原子
o_df_r = original_data.drop(columns='订单详情').join(original_data['订单详情'].str.split('\n',expand=True).stack().reset_index(level=1, drop=True).rename("详情"))


o_df_r = pd.concat([o_df_r[['订单号', '收件人','联系方式','收货地址', '用户信息','下单时间','order_type']], o_df_r['详情'].str.extract(
"""
(?P<商品名>.+?)(?=（规格[：:])（
规格[：:](?P<规格>.+?)(?=[，,]商品ID[：:])
[，,]商品ID[：:](?P<商品ID>.+)）\sX(?P<数量>\s\d+)""",flags=re.X)],axis=1)
o_df_r['发货商ID'] = o_df_r['商品ID'].str[:5]
o_df_r['SPUID'] = o_df_r['商品ID'].str[:-6]
o_df_r.reset_index(drop=True,inplace=True)


o_df_r = pd.merge(o_df_r,sku_goods[['商品ID','单价']],on="商品ID")
o_df_r['数量'] = o_df_r['数量'].astype(int)
o_df_r['总价'] = o_df_r['数量'] * o_df_r['单价']


o_df_r.to_excel(r"C:\Users\qhj01\Desktop\ttt.xlsx")


In [ ]:
data = data.replace("三金美肤面膜 缓解过敏 修复护理|三金美肤面膜 维稳修护 强化屏障", "三金护肤面膜 维稳修护 强化屏障", regex=True)
        patt = '三金补水面膜 镇静维稳 深层补水\\(补水\\)|三金美肤面膜 美白功效 健康肤色\\(美肤\\)|三金护肤面膜 维稳修护 强化屏障\\(护肤\\)|三金水光针\\(水光针\\)'
        data['商品名'] = data['订单详情'].str.extract('(?P<商品名>%s)' % patt, expand=False).str.strip()

In [4]:
# 提取数据
df_t = pd.concat([f_df_r[['订单号', '收件人','联系方式','收货地址', '用户信息','下单时间','order_type']], f_df_r['详情'].str.extract(
"""
(?P<商品名>.+?)(?=（规格[：:])（
规格[：:](?P<规格>.+?)(?=[，,]商品ID[：:])
[，,]商品ID[：:](?P<商品ID>.+)）\sX(?P<数量>\s\d+)""",flags=re.X)],axis=1)
df_t['发货商ID'] = df_t['商品ID'].str[:5]
df_t['SPUID'] = df_t['商品ID'].str[:-6]
df_t.reset_index(drop=True,inplace=True)
# df_t['数量'] = df_t['数量'].astype(str)
# df_t = df_t[['商品名',"发货商ID",'SPUID','商品ID','规格','数量','订单号', '用户信息','下单时间']]

In [12]:
len(f_df_r)

10

In [5]:
# 可发货分类
def tt(row):
    r = row.copy()
#     print(r['数量'])
    s = '；'.join(r['规格'] + " X " +r['数量'])
    r.iloc[0]['数量'] = r['数量'].astype(int).sum()
    r.iloc[0]['规格'] = s
#     print(row)
    print("="*30)
    return r.iloc[0]
    
df_t.groupby(['发货商ID','SPUID']).apply(tt)

订单号  收件人         联系方式  \
发货商ID SPUID                                                 
S0001 S0001P0005C0001  4009762104869953  左泽郡  18774979616   
      S0001P0005C0002  4009762104869953  左泽郡  18774979616   
      S0001P0007C0001  4009762104869953  左泽郡  18774979616   
S0002 S0002P0001C0001  4009762104869953  左泽郡  18774979616   
      S0002P0001C0002  4009762104869953  左泽郡  18774979616   

                                      收货地址  \
发货商ID SPUID                                  
S0001 S0001P0005C0001  湖南省长沙市开福区北桥新村11栋502   
      S0001P0005C0002  湖南省长沙市开福区北桥新村11栋502   
      S0001P0007C0001  湖南省长沙市开福区北桥新村11栋502   
S0002 S0002P0001C0001  湖南省长沙市开福区北桥新村11栋502   
      S0002P0001C0002  湖南省长沙市开福区北桥新村11栋502   

                                                                    用户信息  \
发货商ID SPUID                                                                
S0001 S0001P0005C0001  下单人：左泽郡\n手机：18774979616\n收货地址：湖南省长沙市开福区北桥新村11栋502   
      S0001P0005C0002  下单人：左泽郡\n手机：18774979616\n收货地址：湖南省长沙市开福区北桥新村11栋502   
      S0001P0007C0001  下单人：左泽郡\n手机：18774979616\n收货地址：湖南省长沙市开福区北桥新村11栋502   
S0002 S0002P0001C0001  下单人：左泽郡\n手机：18774979616\n收货地址：湖南省长沙市开福区北桥新村11栋502   
      S0002P0001C0002  下单人：左泽郡\n手机：18774979616\n收货地址：湖南省长沙市开福区北桥新村11栋502   

                                      下单时间 order_type            商品名  \
发货商ID SPUID                                                            
S0001 S0001P0005C0001  2019-12-17 09:12:04   original       蓝牙音箱AP06   
      S0001P0005C0002  2019-12-17 09:12:04   original      颈戴式运动蓝牙耳机   
      S0001P0007C0001  2019-12-17 09:12:04   original  爱玛电动车TDT1067Z   
S0002 S0002P0001C0001  2019-12-17 09:12:04   original      iPhone 11   
      S0002P0001C0002  2019-12-17 09:12:04   original  iPhone 11 Pro   

                                                  规格                   商品ID  \
发货商ID SPUID                                                                   
S0001 S0001P0005C0001  白 X  4；红 X  6；蓝 X  12；银 X  15  S0001P0005C0001P03E01   
      S0001P0005C0002                         黑 X  8  S0001P0005C0002P02E01   
      S0001P0007C0001                      白/亚黑 X  4  S0001P0007C0001P01E01   
S0002 S0002P0001C0001         紫 64G X  5；白 128G X  1  S0002P0001C0001P06E01   
      S0002P0001C0002        金 256G X  3；绿 256G X  3  S0002P0001C0002P06E01   

                       数量  发货商ID            SPUID  
发货商ID SPUID                                        
S0001 S0001P0005C0001  37  S0001  S0001P0005C0001  
      S0001P0005C0002   8  S0001  S0001P0005C0002  
      S0001P0007C0001   4  S0001  S0001P0007C0001  
S0002 S0002P0001C0001   6  S0002  S0002P0001C0001  
      S0002P0001C0002   6  S0002  S0002P0001C0002

In [27]:
df_t.groupby(['发货商ID','SPUID']).apply(tt).to_excel(r"C:\Users\qhj01\Desktop\ttt.xlsx")

In [11]:
stop

NameError: name 'stop' is not defined

In [ ]:
l = []
for order in qm.gen_orders("free",order_time='today'):
    l.append(order)

In [ ]:
pd.DataFrame(l).to_excel(r"C:\Users\qhj01\Desktop\test.xlsx")

In [ ]:
# 正式
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

# 测试



In [ ]:
sku_goods = pd.read_excel(r'D:\奇货居\素材\商城图片素材\商品信息.xlsx',sheet_name='商品详情') \
                [['商品ID','发货商ID','goods_type','规格','单位', '市场价','售价','商品名简称']] \
                .rename(columns={"售价":"单价","商品名简称":"商品名"}).fillna({"goods_type":"original"})

free_goods = sku_goods[sku_goods['goods_type']=='free']

order_kargs = {
    "order_status":1,
    "order_time":"yesterday"
}

yd_data = qm.ext_order(order_kargs,sku_goods,free_goods)

order_kargs = {
    "order_status":1,
    "order_time":"today"
}
to_data = qm.ext_order(order_kargs,sku_goods,free_goods)

In [ ]:

def uprow(row,):
    # 判断是否在已经 发过此物
    if  conn.sismember('order:old' ,row['订单号']):
        row['订单号'] = None
        
    return row





data = pd.concat([to_data.apply(uprow,axis=1), #
                  yd_data.apply(uprow,axis=1)])
data.dropna(subset=['订单号'],inplace=True)
data.drop_duplicates(subset='订单号',inplace=True)

In [ ]:
len(data)

In [ ]:

save_dir = r"D:\Downloads\QHJ_MALL"
save = SaveXl(save_dir)
save(data,"订单")
%run D:\往期\QHJ\ZERO\app\process_order.py


In [ ]:


stop


In [ ]:
spp_path =  os.path.join(r"D:\奇货居\素材\商城图片素材","发货商详情.xlsx")
spp_data = pd.read_excel(spp_path)

In [ ]:
spp_data

In [ ]:
stop

In [ ]:
d = dict(zip(spp_data['发货商ID'],spp_data['发货商'],))

d

In [ ]:
conn.keys('ORDER:NEW:TEMP*')

In [ ]:
conn = redis.Redis(**REDIS_MALL_ORDER_DIC)

conn.keys('ORDER:NEW:TEMP*')
src_new = 'ORDER:NEW:TEMP:S0003'
dst_new = src_new.replace(":TEMP","")
conn.sunionstore( dst_new,dst_new,src_new)
conn.delete(src_new)

In [ ]:
# conn.sunionstore('ORDER:NEW',"ORDER:NEW_TEMP",'ORDER:NEW')

# conn.sadd("ORDER:NEW","zero")
# conn.sadd("ORDER:WB","zero")
# conn.sadd("ORDER:NEW_TEMP","zero")